## VarQITE Error

In [ ]:
from jax import grad, jit
# import jax.numpy as jnp
import numpy as jnp

import os
import warnings

import numpy as np
import scipy as sp
from scipy.linalg import expm

from qiskit import Aer
from qiskit.circuit.library import EfficientSU2, RealAmplitudes
from qiskit.opflow import StateFn, MatrixOp, CircuitOp, X, I, PauliOp, Z, Y, SummedOp
from qiskit.opflow.gradients import NaturalGradient
# from qiskit.aqua.operators import StateFn, MatrixOp, CircuitOp, X, I, PauliOp, Z, Y, SummedOp
# from qiskit.aqua.operators.gradients import NaturalGradient
from qiskit.opflow.evolutions.varqtes.varqite import VarQITE

np.random.seed = 2

In [2]:
# Set Hamiltonian
# Hamiltonian = (Z ^ X) # works
# Hamiltonian = SummedOp([(Z ^ X), 0.3 *(Y ^ Y), (Y ^ I)]).reduce() # works
Hamiltonian = SummedOp([(Z ^ X), 3. * (Y ^ Y), (Z ^ X), (I ^ Z), (Z ^ I)]).reduce() #works
# Hamiltonian = (Y ^ I)
# Set time and time_steps 
time = 1
time_steps = 10

In [3]:
Hamiltonian.to_matrix()

array([[ 2.+0.j,  2.+0.j,  0.+0.j, -3.+0.j],
       [ 2.+0.j,  0.+0.j,  3.+0.j,  0.+0.j],
       [ 0.+0.j,  3.+0.j,  0.+0.j, -2.+0.j],
       [-3.+0.j,  0.+0.j, -2.+0.j, -2.+0.j]])

In [4]:
# Helper Function computing the inner product
def inner_prod(x, y):
    return np.matmul(np.conj(np.transpose(x)), y)

In [5]:
# Set Ansatz and initial parameters
ansatz = EfficientSU2(2, reps=1, entanglement='linear')
parameters = ansatz.ordered_parameters
init_param_values = np.zeros(len(ansatz.ordered_parameters))
for i in range(ansatz.num_qubits):
    init_param_values[-(ansatz.num_qubits + i + 1)] = np.pi / 2
    
# ansatz = RealAmplitudes(Hamiltonian.num_qubits, reps=1, entanglement='linear')
# parameters = ansatz.ordered_parameters
# init_param_values = np.zeros(len(ansatz.ordered_parameters))
# for i in range(ansatz.num_qubits):
#     init_param_values[-(i + 1)] = np.pi / 2


### Analytic Calculations

In [6]:
def analytic_error(state, H, et_ket=None):
    if et_ket is None:
        et_ket = np.zeros(len(state))


    energy = inner_prod(state, np.dot(H, state))
    # This is the exact derivative see Eq. (2)
    dt_analytic = energy * state - np.dot(H, state)
    # Now we choose et_ket and use Eq. (2) to compute |dt_state
    dt_state = dt_analytic + et_ket
    # <dt|dt>
    et = inner_prod(dt_state, dt_state) 
#     print('dtdt', inner_prod(dt_state, dt_state))
    #<H^2>
    et += inner_prod(state, np.matmul(np.matmul(H, H), state))
    # <H>^2
#     print('<H^2>', inner_prod(state, np.matmul(np.matmul(H, H), state)))
    et -= energy ** 2
#     print('e^2', energy **2)
    # 2Re<dt|H|state>
#     print('Re<dt|H|state>', np.real(inner_prod(dt_state, np.matmul(H, state))))
    et += 2 * np.real(inner_prod(dt_state, np.matmul(H, state)))
    # 2Re(<dt|state>)*E_t
#     et -= 2 * np.real(inner_prod(dt_state, state)) * energy
                     
#     print('Term in question 2Re(<dt|state>)*E_t  not included', 2 * np.real(inner_prod(dt_state, state) * energy))
    
    print('Grad error', np.round(np.sqrt(et), 3))
    return np.sqrt(et), dt_state
    
    

In [7]:
num_qubits = Hamiltonian.num_qubits
H = Hamiltonian.to_matrix()

# Propagation Unitary
evolution_op = lambda t: expm(-1 * H * t)
# Initial State
init_state = StateFn(ansatz).assign_parameters(dict(zip(parameters, init_param_values))).eval().primitive

target_state = lambda t: np.dot(evolution_op(t), init_state)/\
                         np.sqrt(inner_prod(init_state, inner_prod(evolution_op(2*t), init_state)))

dt_target_state = lambda t: np.dot(-1 * H, target_state(t)) - \
                            np.dot(evolution_op(t), init_state) * 0.5 / \
                            (inner_prod(init_state,
                                        inner_prod(evolution_op(2*t), init_state))) ** (3/2) * \
                            inner_prod(init_state, inner_prod(-2*H, inner_prod(evolution_op(2*t),
                                                                          init_state)))

error = 0
prepared_state = target_state(0)
for j in range(0, time_steps):
    et_ket = np.ones(2 ** num_qubits) / np.sqrt(2 ** num_qubits) * j * -1e-2

    et, dt_prepared_state = analytic_error(prepared_state, H, et_ket=et_ket)
    # Euler State propagation
    prepared_state += time/time_steps * dt_prepared_state
    # Compute Error
#     error += time/time_steps * et * (1 + 2 * time/time_steps*np.linalg.norm(H))**(
#                  time-j*time/time_steps)
    error += time/time_steps * np.sqrt(np.linalg.norm(et_ket)) * (1 + 2 * time/time_steps*sp.linalg.norm(H))**(
                 time-j*time/time_steps)

    print('error bound for time', np.round(time/time_steps*(j+1), 3), np.round(error, 3))

# sqrt of the l2-norm of the distance between the perturbed, prepared state and the exat, target state
print('True error ', np.sqrt(np.linalg.norm(prepared_state - target_state(time))))


Grad error 0j
error bound for time 0.1 0.0
Grad error (1.653+0j)
error bound for time 0.2 0.023
Grad error (2.159+0j)
error bound for time 0.3 0.053
Grad error (0.926+0j)
error bound for time 0.4 0.086
Grad error (0.666+0j)
error bound for time 0.5 0.122
Grad error (0.544+0j)
error bound for time 0.6 0.157
Grad error (0.505+0j)
error bound for time 0.7 0.193
Grad error (0.505+0j)
error bound for time 0.8 0.228
Grad error (0.522+0j)
error bound for time 0.9 0.262
Grad error (0.544+0j)
error bound for time 1.0 0.295
True error  0.14599179269403711


### Analytic + Euler

In [8]:
def analytic_euler_error(state, H, et_ket=None, dt_state=None):
    if et_ket is None:
        et_ket = np.zeros(len(state))


    energy = inner_prod(state, np.dot(H, state))
    # This is the exact derivative see Eq. (2)
    dt_analytic = energy * state - np.dot(H, state)
    # Now we choose et_ket and use Eq. (2) to compute |dt_state
    if dt_state is None:
        dt_state = dt_analytic + et_ket
    # <dt|dt>
    et = inner_prod(dt_state, dt_state) 
#     print('dtdt', inner_prod(dt_state, dt_state))
    #<H^2>
    et += inner_prod(state, np.matmul(np.matmul(H, H), state))
    # <H>^2
#     print('<H^2>', inner_prod(state, np.matmul(np.matmul(H, H), state)))
    et -= energy ** 2
#     print('e^2', energy **2)
    # 2Re<dt|H|state>
#     print('Re<dt|H|state>', np.real(inner_prod(dt_state, np.matmul(H, state))))
    et += 2 * np.real(inner_prod(dt_state, np.matmul(H, state)))
    # 2Re(<dt|state>)*E_t
#     et -= 2 * np.real(inner_prod(dt_state, state)) * energy
                     
#     print('Term in question 2Re(<dt|state>)*E_t  not included', 2 * np.real(inner_prod(dt_state, state) * energy))
    
    print('Grad error', np.round(np.sqrt(et), 3))
    return np.sqrt(et), dt_state
    

In [9]:
num_qubits = Hamiltonian.num_qubits
H = Hamiltonian.to_matrix()

# Propagation Unitary
evolution_op = lambda t: expm(-1 * H * t)
# Initial State
init_state = StateFn(ansatz).assign_parameters(dict(zip(parameters, init_param_values))).eval().primitive

target_state = lambda t: np.dot(evolution_op(t), init_state)/\
                         np.sqrt(inner_prod(init_state, inner_prod(evolution_op(2*t), init_state)))

dt_target_state = lambda t: np.dot(-1 * H, target_state(t)) - \
                            np.dot(evolution_op(t), init_state) * 0.5 / \
                            (inner_prod(init_state,
                                        inner_prod(evolution_op(2*t), init_state))) ** (3/2) * \
                            inner_prod(init_state, inner_prod(-2*H, inner_prod(evolution_op(2*t),
                                                                          init_state)))

error = 0
prepared_state = target_state(0)
target_euler_state = target_state(0)
for j in range(0, time_steps):
#     et_ket = np.ones(2 ** num_qubits) / np.sqrt(2 ** num_qubits) * j * -1e-2

    et, dt_prepared_state = analytic_euler_error(prepared_state, H, et_ket=et_ket, dt_state = dt_target_state(j * time/time_steps))
    # Euler State propagation
    prepared_state += time/time_steps * dt_prepared_state
    # Compute Error
#     error += time/time_steps * et * (1 + 2 * time/time_steps*np.linalg.norm(H))**(
#                  time-j*time/time_steps)
    error += time/time_steps * np.sqrt(np.linalg.norm(et_ket)) * (1 + 2 * time/time_steps*sp.linalg.norm(H))**(
                 time-j*time/time_steps)

    print('error bound for time', np.round(time/time_steps*(j+1), 3), np.round(error, 3))
    target_euler_state += time/time_steps * dt_target_state(j * time/time_steps)
    print('Error trained and true `Euler` state', np.sqrt(np.linalg.norm(prepared_state - target_euler_state)))

# sqrt of the l2-norm of the distance between the perturbed, prepared state and the exat, target state
print('True error ', np.sqrt(np.linalg.norm(prepared_state - target_state(time))))

Grad error 0j
error bound for time 0.1 0.076
Error trained and true `Euler` state 0.0
Grad error 1.515j
error bound for time 0.2 0.146
Error trained and true `Euler` state 0.0
Grad error 2.666j
error bound for time 0.3 0.21
Error trained and true `Euler` state 0.0
Grad error 2.97j
error bound for time 0.4 0.267
Error trained and true `Euler` state 0.0
Grad error 3.058j
error bound for time 0.5 0.32
Error trained and true `Euler` state 0.0
Grad error 3.095j
error bound for time 0.6 0.368
Error trained and true `Euler` state 0.0
Grad error 3.112j
error bound for time 0.7 0.411
Error trained and true `Euler` state 0.0
Grad error 3.119j
error bound for time 0.8 0.451
Error trained and true `Euler` state 0.0
Grad error 3.121j
error bound for time 0.9 0.487
Error trained and true `Euler` state 0.0
Grad error 3.121j
error bound for time 1.0 0.52
Error trained and true `Euler` state 0.0
True error  0.4702073728354985


### Numpy Calculations

In [10]:
def dt_params(a, c, regularization=None):
    if regularization:
        # If a regularization method is chosen then use a regularized solver to
        # construct the natural gradient.
        nat_grad = NaturalGradient._regularized_sle_solver(
            a, c, regularization=regularization)
    else:
#         nat_grad = np.linalg.lstsq(a, c, rcond=None)[0]
#         if np.linalg.norm(nat_grad) < 1e-8:
#             nat_grad = NaturalGradient._regularized_sle_solver(a,
#                                                                c,
#                                                                regularization='perturb_diag')
#             warnings.warn(r'Norm of the natural gradient smaller than $1e^{-8}$ use '
#                           r' `perturb_diag` regularization.')
#         if np.linalg.norm(nat_grad) > 1e-4:
#             nat_grad = NaturalGradient._regularized_sle_solver(a,
#                                                                c,
#                                                                regularization='ridge')
#             warnings.warn(r'Norm of the natural gradient bigger than $1e^{3}$ use '
#                           r' `ridge` regularization.')
        try:
#             # Try to solve the system of linear equations Ax = C.
            nat_grad = np.linalg.solve(a, c)
        except np.linalg.LinAlgError:  # singular matrix
            print('Singular matrix lstsq solver required')
            nat_grad, resids, _, _ = np.linalg.lstsq(a, c)
            print('Residuals', resids)
    return np.real(nat_grad)

In [11]:
def numpy_error(a, c, dt_weights, H, psi_t, grad):
    psi_t_dot = np.transpose(np.matmul(grad, dt_weights))
    
    dtdt = np.matmul(np.transpose(dt_weights), np.matmul(a, dt_weights))[0][0] 
    print('dtdt alternative', np.linalg.norm(psi_t_dot)**2)
#     print('a', a)
    et = dtdt
    print(et)
#     print('H^2', np.matmul(H, H))
    h_squared = inner_prod(psi_t, np.matmul(np.matmul(H, H), psi_t))
    et = np.add(et, h_squared)

    energy = inner_prod(psi_t, np.matmul(H, psi_t))
    et -= energy ** 2
    
    psi_t_dot_analytic = np.reshape(energy * psi_t - np.dot(H, psi_t), np.shape(psi_t_dot))
    
    # np.matmul(energy*np.eye(len(state))-H, state)
    eth = energy*np.eye(len(psi_t))-H
#     dt_alt = -2 * np.real(inner_prod( np.matmul(grad, dt_weights), np.matmul(eth, state)))
    dt = 2 * inner_prod(c, dt_weights)[0][0] 
    et += dt
    print('psi dot', psi_t_dot)
    print('energy * psi_t - np.matmul(H, psi_t)', energy * psi_t - np.matmul(H, psi_t))
    et_other = psi_t_dot  - (energy * psi_t - np.matmul(H, psi_t))
    et_analytic = inner_prod(psi_t_dot_analytic - psi_t_dot, psi_t_dot_analytic - psi_t_dot) 
#     print('et analytic', et_analytic)

#     overlap = 2*np.real(np.dot(np.transpose(np.conj(dt_state)), state))
#     print('2E_t <dt_state|state>', energy * overlap)
    
#     if et <0:
    print('-----------------------------------------')
    print('dt_weights', dt_weights)
    print('grad ', grad)
    print('norm c', sp.linalg.norm(c))
#     #TODO eigenvalues
    print('eigvalues a', np.linalg.eig(a)[0])
    print('singular values a', np.linalg.svd(a)[1])
    print('cond a', np.linalg.cond(a))
#         print('alterantive 2 Re dtH', dt_alt)
    print('2 Re dt<H>', np.round(dt, 5))
    print('<dt_state|dt_state>', np.round(dtdt, 5))
    print('h^2', np.round(h_squared, 5))
    print('Energy^2', np.round(energy ** 2, 5))
#     print('|adtw+c|^2', np.linalg.norm(np.matmul(a, dt_weights) + c))
#     print('rank A', np.linalg.matrix_rank(a))
    print('Grad error', np.real(np.sqrt(et) ))
    print('shape dtpsi_t', np.shape(psi_t_dot))
    print('shape (E-H)psi_t ', np.shape(energy * psi_t - np.matmul(H, psi_t)))
    print('norm of Grad Error = dtpsi_t - (E-H)psi_t ', np.linalg.norm(et_other))
    print('-----------------------------------------')


    # TODO discuss capping
    return np.sqrt(et) 
#     return np.linalg.norm(et_other)

In [12]:
print('Warning: Numpy Calculations are only compatible with SU2 depth 1 - else hard-coded changes needed.')

def ry(theta):
    return jnp.array([[jnp.cos(theta/2.), -1*jnp.sin(theta/2)], [jnp.sin(theta/2),
                                                                 jnp.cos(theta/2)]])

def rz(theta):
    return jnp.array([[jnp.exp(-1j * theta / 2.), 0], [0, jnp.exp(1j * theta / 2.)]])


def ryry(alpha, beta):
    return jnp.kron(ry(alpha), ry(beta))

def rzrz(alpha, beta):
    return jnp.kron(rz(alpha), rz(beta))

cx = jnp.array([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]])
i = jnp.eye(2)
y = jnp.array([[0, -1j], [1j, 0]])
z = jnp.array([[1, 0], [0, -1]])
iy = -0.5j * jnp.kron(i, y)
yi = -0.5j * jnp.kron(y, i)
iz = -0.5j * jnp.kron(i, z)
zi = -0.5j * jnp.kron(z, i)

init = jnp.array([1, 0, 0, 0])

def state_fn(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec

def state0(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec[0]

def state1(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec[1]

def state2(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec[2]

def state3(params):
    vec = jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],
                                                                                params[0]),
                                                                           init))))
    return vec[3]


def A(vec, gradient):
    vec = np.reshape(vec, (len(vec), 1))
    a = np.real(inner_prod(gradient, gradient))
#     print('a carrying part', np.round(a, 3))
    a = np.subtract(a, np.real(np.matmul(inner_prod(gradient, vec),
                    np.transpose(np.conj(inner_prod(gradient, vec))))))
    print('a phase fix', np.round(np.real(np.matmul(inner_prod(gradient, vec),
                    np.transpose(np.conj(inner_prod(gradient, vec))))), 3))
    
    w, v = np.linalg.eig(a)
    if not all(ew >= -1e-8 for ew in w):
        raise Warning('The underlying metric has ein Eigenvalue < ', -1e-8, '. '
                      'Please use a regularized least-square solver for this problem.')
    if not all(ew >= 0 for ew in w):
        w = [max(0, ew) for ew in w]
        a = v @ np.diag(w) @ np.linalg.inv(v)
    if np.any([[np.abs(np.imag(a_item)) > 1e-8 for a_item in a_row] for a_row in a]):
        raise Warning('The imaginary part of the gradient are non-negligible.')
    return np.real(a)

def C(vec, gradient, h):
    vec = np.reshape(vec, (len(vec), 1))
    c = np.real(inner_prod(gradient, np.matmul(h, vec)))
    if any(np.abs(np.imag(c_item)) > 1e-8 for c_item in c):
        raise Warning('The imaginary part of the gradient are non-negligible.')
    return np.real(c)

def grad0(params):
    try:
        return grad(state0)(params)
    except Exception:
        return grad(state0, holomorphic=True)(jnp.complex64(params))
def grad1(params):
    try:
        return grad(state1)(params)
    except Exception:
        return grad(state1, holomorphic=True)(jnp.complex64(params))
def grad2(params):
    try:
        return grad(state2)(params)
    except Exception:
        return grad(state2, holomorphic=True)(jnp.complex64(params))
def grad3(params):
    try:
        return grad(state3)(params)
    except Exception:
        return grad(state3, holomorphic=True)(jnp.complex64(params))
    
def grad_fn(params):
    # num_params x dim vec
    g = []
    # g.append(jnp.dot(ryry(params[3], params[2]), jnp.dot(cx, jnp.dot(iy, jnp.dot(ryry(params[1],
    #                                                                                   params[0]),
    #                                                                init)))))
    # g.append(jnp.dot(ryry(params[3], params[2]), jnp.dot(cx, jnp.dot(yi, jnp.dot(ryry(params[1],
    #                                                                                   params[0]),
    #                                                                              init)))))
    # g.append(jnp.dot(iy, jnp.dot(ryry(params[3], params[2]), jnp.dot(cx, jnp.dot(ryry(params[1],params[0]),
    #                                                                              init)))))
    # g.append(jnp.dot(yi, jnp.dot(ryry(params[3], params[2]),
    #                              jnp.dot(cx, jnp.dot(ryry(params[1], params[0]),
    #                                                  init)))))
    g.append(jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(iy, jnp.dot(ryry(params[1],
                                                                                      params[0]),
                                                                   init))))))
    g.append(jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]),  jnp.dot(yi, jnp.dot(ryry(params[1],
                                                                                      params[0]),
                                                                                 init))))))
    g.append(jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(iz, jnp.dot(ryry(
                      params[1],
                                                                                      params[0]),
                                                                   init))))))
    g.append(jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]),  jnp.dot(zi, jnp.dot(ryry(
                      params[1],
                                                                                      params[0]),
                                                                                 init))))))
    g.append(jnp.dot(jnp.dot(jnp.dot(rzrz(params[7], params[6]), iy), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]), jnp.dot(ryry(params[1],params[0]),
                                                                                 init)))))
    g.append(jnp.dot(jnp.dot(jnp.dot(rzrz(params[7], params[6]), yi), ryry(params[5], params[4])),
                  jnp.dot(cx, jnp.dot(rzrz(params[3],  params[2]),  jnp.dot(ryry(params[1], params[0]),
                                                     init)))))
    g.append(jnp.dot(iz, jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                                 jnp.dot(cx, jnp.dot(rzrz(params[3], params[2]),
                                                     jnp.dot(ryry(params[1], params[0]),
                                                             init))))))
    g.append(jnp.dot(zi, jnp.dot(jnp.dot(rzrz(params[7], params[6]), ryry(params[5], params[4])),
                                 jnp.dot(cx, jnp.dot(rzrz(params[3], params[2]),
                                                     jnp.dot(ryry(params[1], params[0]),
                                                             init)))))
             )
    return g



In [13]:
init_param_values = np.zeros(len(ansatz.ordered_parameters))
for i in range(ansatz.num_qubits):
    init_param_values[-(ansatz.num_qubits + i + 1)] = np.pi / 2
params = init_param_values
error = 0

for j in range(time_steps):

    # dim vec x num_params
#     gradient = [grad0(params), grad1(params), grad2(params), grad3(params)]
#     gradient = np.array([[complex(item) for item in g] for g in gradient]).astype(np.complex)
    gradient = grad_fn(params)
    gradient = np.array([[complex(item) for item in g] for g in gradient]).astype(np.complex)
    gradient = np.transpose(gradient)
    
    print('Parameters', params)
    state = state_fn(params)
    state = np.array([complex(s) for s in state]).astype(np.complex)
#     print('state', state)
    # QFI/4
    metric = A(state, gradient)
    # Re⟨dψ(ω)/dω|H|ψ(ω)
    c_grad = C(state, gradient, np.transpose(H))
    
#     print('c', np.transpose(np.round((2)*c_grad, 3)))
#     print('metric', np.round(metric, 3))

    # dω/dt
    dt_weights = dt_params(metric, (-1)*c_grad, 'ridge')
    res = np.linalg.norm(np.matmul(metric, dt_weights)+c_grad)
    print('|adtw+c|', res)
    overlap = 2*np.real(inner_prod(np.matmul(gradient, dt_weights), state))
#     print('overlap', overlap)

    # Gradient error approximation
    et = numpy_error(metric, c_grad, dt_weights, np.transpose(H), state, gradient)
    
    if np.imag(et) > 1e-5:
        raise Warning('The erro of this step is imaginary. Thus, the SLE underlying McLachlan was not solved well. '\
                      'The residuum of the SLE is ', res)

    # Add current gradient error to the error bound
    
    # TODO discuss (25) index of time step or time? I think indices
#     error += time/time_steps * et * (1 + 2 * time/time_steps*np.linalg.norm(H))**(
#              time-(j*time/time_steps))
    error += time/time_steps * et * (1 + 2 * time/time_steps*np.linalg.norm(H, 2))**(
             time_steps-j-1)
#     error += time/time_steps * et * (1 + 2 * time/time_steps*np.linalg.norm(H, np.inf))**(
#              time_steps-j)
#     error += time/time_steps * et
#     print('factor', (1 + 2 * time/time_steps*sp.linalg.norm(H))**(
#              time_steps-j))
#     print('factor', (1 + 2 * time/time_steps*sp.linalg.norm(H))**(
#              time_steps-j))

    print('Inf Norm H', np.linalg.norm(H, np.inf))
#     error += time/time_steps *  (et + 2 * np.linalg.norm(H, np.inf))

    params += time/time_steps * np.reshape(dt_weights, np.shape(params))
#     print('Parameters', params)
        
    print('error bound for time', np.round(time/time_steps*(j+1), 3), np.round(error, 4))

    # sqrt of the l2-norm of the distance between the perturbed, prepared state and the exat, target state
    print('True error ', np.linalg.norm(state_fn(params) - target_state(time/time_steps*(j+1))))

Parameters [0.         0.         0.         0.         1.57079633 1.57079633
 0.         0.        ]
a phase fix [[0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.25 0.   0.   0.   0.  ]
 [0.   0.   0.25 0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]]
|adtw+c| 0.007417184800450264
dtdt alternative 18.89980058025972
18.899800580259722
psi dot [[-0.49880914+4.37199861e-03j -2.49561401-4.33680869e-19j
  -0.4952213 +4.33680869e-19j  3.48964445-4.37199861e-03j]]
energy * psi_t - np.matmul(H, psi_t) [-0.5+0.j -2.5+0.j -0.5+0.j  3.5+0.j]
-----------------------------------------
dt_weights [[ 5.98167062]
 [ 2.99442315]
 [ 0.        ]
 [ 0.        ]
 [ 1.98806088]
 [ 2.99442315]
 [-0.008744  ]
 [-0.008744  ]]
grad  [[ 0.25+0.j   -0.25+0.j    0.  -0.25j  0.  -0.25j -0.25+0.j   -0.25+0.j
   0

|adtw+c| 0.005163097506274526
dtdt alternative 0.4967956992320473
0.4967818802243839
psi dot [[ 0.00555967+0.00073293j -0.5335962 -0.01272933j  0.28558871+0.0020652j
  -0.36098574-0.00119392j]]
energy * psi_t - np.matmul(H, psi_t) [ 0.01047217+0.00146287j -0.54147328-0.00709979j  0.28506483-0.00336898j
 -0.3662512 -0.00136242j]
-----------------------------------------
dt_weights [[ 0.53488876]
 [-0.4010212 ]
 [-0.01029508]
 [-0.01375429]
 [-0.6973016 ]
 [ 0.79686942]
 [-0.00426308]
 [-0.00462348]]
grad  [[ 2.05042370e-01-0.00161497j -4.03746134e-01-0.0015335j
   1.99502842e-03+0.12102883j  1.87825210e-03-0.3481341j
   1.45897041e-01-0.00184522j -2.06097946e-01-0.00706645j
   1.37151798e-03-0.20073213j  1.37151798e-03-0.20073213j]
 [-2.86478061e-01-0.00616495j -1.59235173e-01-0.00680034j
   5.40622050e-03-0.21657202j  1.84051731e-03+0.06814006j
   2.00666259e-01+0.00532197j -3.81760767e-01-0.01215713j
   1.02661281e-03-0.1459051j  -1.02661281e-03+0.1459051j ]
 [-3.53699457e-01+0.005490

|adtw+c| 0.0008221852590923821
dtdt alternative 0.011923737422375444
0.011922298134232524
psi dot [[-0.01590466+3.88903689e-04j -0.07111132-1.44376682e-03j
   0.04636384+1.20172748e-03j -0.06678825+6.70465814e-05j]]
energy * psi_t - np.matmul(H, psi_t) [-0.01558412+0.00323097j -0.07222904+0.0030138j   0.04615697-0.00231678j
 -0.06747426+0.00183098j]
-----------------------------------------
dt_weights [[ 0.06099515]
 [-0.05649826]
 [-0.00108501]
 [-0.00335607]
 [-0.10780218]
 [ 0.1167954 ]
 [-0.00242053]
 [ 0.00080727]]
grad  [[ 1.96593774e-01-0.00130302j -3.88870365e-01-0.00211369j
   1.63265080e-03+0.14212091j  2.10479801e-03-0.31186009j
   2.03576983e-01-0.00127087j -2.38957394e-01-0.00853075j
   1.49767550e-03-0.19580992j  1.49767550e-03-0.19580992j]
 [-3.03759428e-01-0.00612074j -1.37668591e-01-0.00581012j
   5.36455691e-03-0.24404749j -2.73792302e-04+0.14758292j
   1.95753713e-01+0.0049248j  -3.36027975e-01-0.0110823j
   2.29263619e-03-0.20356804j -2.29263619e-03+0.20356804j]
 [-

### Variational Calculations

In [14]:
init_param_values = np.zeros(len(ansatz.ordered_parameters))
for i in range(ansatz.num_qubits):
    init_param_values[-(ansatz.num_qubits + i + 1)] = np.pi / 2
op = ~StateFn(Hamiltonian) @ StateFn(ansatz)
op = time * op




# This used
time_steps = 10

backend = Aer.get_backend('statevector_simulator')
init_param_values = np.zeros(len(ansatz.ordered_parameters))
for i in range(ansatz.num_qubits):
    init_param_values[-(ansatz.num_qubits + i + 1)] = np.pi / 2
op = ~StateFn(Hamiltonian)@StateFn(ansatz)
backend = Aer.get_backend('statevector_simulator')
# op = ~StateFn(ansatz)@Hamiltonian@StateFn(ansatz)
op = time * op
approx_time_evolved_state = VarQITE(parameters=parameters,
                                    grad_method='lin_comb',
                                    init_parameter_values=init_param_values, num_time_steps=time_steps,
                                    regularization='ridge', backend=backend,
                                    error_based_ode=False,
                                    snapshot_dir=os.path.join('dummy')).convert(op)
# approx_time_evolved_state = VarQITE(parameters=parameters, get_error=True,
#                                     grad_method='lin_comb',
#                                     init_parameter_values=init_param_values, num_time_steps=time_steps,
#                                     fidelity_to_target=True,
#                                     regularization='perturb_diag'
#                                     ).convert(op)

nat grad result [ 5.98167060e+00  2.99442314e+00 -3.78664601e-15 -2.80150125e-15
  1.98806086e+00  2.99442314e+00 -8.74400861e-03 -8.74400861e-03]
returned et 0.000188957723125327
after try except 0.013746189403806678
Fidelity 1.0
True error 0.0
actual energy (8.881784197001252e-16+0j)
trained_en (8.881784197001252e-16+0j)
nat grad result [ 5.98167060e+00  2.99442314e+00 -3.78664601e-15 -2.80150125e-15
  1.98806086e+00  2.99442314e+00 -8.74400861e-03 -8.74400861e-03]
nat grad result [ 3.43485743  2.01810906 -0.37151033 -0.10713224 -0.63904927  2.64148673
  0.01224183 -0.1825056 ]
returned et 0.03160025494009844
after try except 0.17776460541991604
Fidelity 0.9939509888986188
True error 0.07784382481621809
actual energy (-3.117074989850884+0j)
trained_en (-3.251787591327096-8.673617379884035e-19j)
ode time 0.1
ode parameters [0.5981670598192337, 0.2994423139494295, -3.7866460055294577e-16, -2.801501250158212e-16, 1.769602412827132, 1.8702386407443092, -0.0008744008612636136, -0.00087440

In [ ]:
 [ 1.32724372  0.32633907 -0.02726351 -0.01753228  1.28241228  2.65054732
  0.01726233 -0.02503704]
error bound for time 1.0 (7.89+0j)
True error  0.013091233327682479

In [15]:
import scipy as sp

In [16]:
t = sp.linalg.expm([[0, 1j], [-1j, 0]])

In [17]:
t = t/np.linalg.norm(t)

In [18]:
t

array([[0.56253938+0.j        , 0.        +0.42842671j],
       [0.        -0.42842671j, 0.56253938+0.j        ]])

In [19]:
846.894239 / 38037.43500115013

0.022264756784320303

In [20]:
Fidelity 0.8538344497205992
True error 1.0559715879774545
Error 388083537390.2435 after 1.0
      194041768695

SyntaxError: invalid syntax (<ipython-input-20-0ed8651a948a>, line 1)

In [ ]:
np.exp(2 * np.abs(1) * 5.273163042697021)